In [1]:
import datetime
from sklearn import metrics, model_selection, ensemble
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import KBinsDiscretizer
import xgboost as xgb
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold

C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
try: 
    import optuna
except:
    !pip install optuna
    import optuna

In [3]:
try:
    from pytorch_tabnet.tab_model import TabNetRegressor
    from pytorch_tabnet.pretraining import TabNetPretrainer

except:
    !pip install pytorch-tabnet
    from pytorch_tabnet.tab_model import TabNetRegressor
    from pytorch_tabnet.pretraining import TabNetPretrainer


In [32]:
try:
    train = pd.read_csv("./train_for_crossval.csv", delimiter=",", sep='.')
except:
    train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Kaggle_ML30days/train_for_crossval.csv", delimiter=",", sep='.')

In [33]:
train

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,anomaly_col
0,B,A,A,C,B,B,A,E,E,I,...,0.449762,0.281378,0.566167,0.564149,0.094746,0.347672,0.948445,0.410370,7.200575,Norm
1,A,A,A,C,B,D,A,E,A,I,...,0.562349,0.349834,0.953103,0.808620,0.663709,0.503259,0.692824,0.760255,9.857370,Norm
2,A,B,A,C,B,B,A,E,E,G,...,0.339495,0.365626,0.471769,0.318679,0.249393,0.352284,0.334168,0.265968,8.476475,Norm
3,A,B,A,C,B,B,A,E,C,I,...,0.607722,0.653107,0.555535,0.671799,0.573051,0.391958,-0.072907,0.524188,8.015375,Norm
4,B,A,A,C,B,D,A,E,C,H,...,0.902596,0.892918,0.721439,0.346334,0.119194,0.413215,0.423083,0.264100,10.375199,Norm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,A,A,A,C,B,D,A,D,A,F,...,0.594021,1.032560,0.905694,0.852805,0.809571,0.358780,0.823077,0.865032,8.521849,Anomaly
89996,B,A,A,D,B,B,A,E,E,B,...,0.361353,0.900748,0.468781,0.267659,0.695161,0.421088,0.841941,0.615257,8.988227,Anomaly
89997,B,A,B,C,B,D,A,E,A,F,...,0.705946,0.896323,0.639372,0.598032,0.490381,0.134411,0.764455,0.240164,9.021154,Norm
89998,B,A,A,C,B,D,A,E,G,J,...,0.331300,0.369129,0.601829,0.506245,0.583071,0.429652,0.776692,0.662461,7.701070,Norm


In [46]:
X = train.drop(["target"], axis=1)
y = train["target"] * 100

In [49]:
num_train = X.select_dtypes([int, float])
cat_train = X.select_dtypes(object)

num = list(num_train)
cat = list(cat_train)

idx = [X.columns.get_loc(i) for i in cat]
dims = X[cat].nunique().to_list()

In [53]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in cat:
    X[col] = le.fit_transform(X[col])

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [56]:
X_val, X_test, y_val, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [64]:
unsupervised_model = TabNetPretrainer(
    cat_idxs=idx, cat_dims=dims,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=0.1),
    mask_type='sparsemax' # "sparsemax" entmax
)

unsupervised_model.fit(
    X_train=X_train.values,
    eval_set=[X_val.values],
    pretraining_ratio=0.8,
)
# best_val_0_unsup_loss = 0.82771 lr=0.01
# best_val_0_unsup_loss = 0.77266 lr=0.1

Device used : cpu
epoch 0  | loss: 1.02597 | val_0_unsup_loss: 3.0493  |  0:00:13s
epoch 1  | loss: 0.96176 | val_0_unsup_loss: 3.69506 |  0:00:28s
epoch 2  | loss: 0.96344 | val_0_unsup_loss: 5.55024 |  0:00:44s
epoch 3  | loss: 0.96284 | val_0_unsup_loss: 5.37263 |  0:01:02s
epoch 4  | loss: 0.9637  | val_0_unsup_loss: 5.78898 |  0:01:19s
epoch 5  | loss: 0.964   | val_0_unsup_loss: 6.19711 |  0:01:38s
epoch 6  | loss: 0.96214 | val_0_unsup_loss: 5.46408 |  0:01:55s
epoch 7  | loss: 0.96003 | val_0_unsup_loss: 5.69499 |  0:02:13s
epoch 8  | loss: 0.96086 | val_0_unsup_loss: 5.21576 |  0:02:37s
epoch 9  | loss: 0.96065 | val_0_unsup_loss: 4.53498 |  0:03:09s
epoch 10 | loss: 0.96092 | val_0_unsup_loss: 5.1301  |  0:03:28s

Early stopping occurred at epoch 10 with best_epoch = 0 and best_val_0_unsup_loss = 3.0493
Best weights from best epoch are automatically used!


In [60]:
model = TabNetRegressor(
    cat_idxs=idx, cat_dims=dims,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=0.01),
    scheduler_params={"step_size":10, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='sparsemax' # This will be overwritten if using pretrain model
)

model.fit(
    X_train=X_train.values, y_train=y_train.values.reshape(-1, 1),
    eval_set=[(X_train.values, y_train.values.reshape(-1, 1)), (X_val.values, y_val.values.reshape(-1, 1))],
    eval_name=['train', 'valid'],
    eval_metric=['rmse'],
    from_unsupervised=unsupervised_model
)



Device used : cpu


KeyError: 48795